In [129]:
# Tasks
# 1-) Nifty50 - current price is 14900 (then count strikes from 14400-15400) if price will be 13900 then it should count 
# (13400-14400) similarly if it 15900 it
# should count strikes (15400-16400) like 500 up and 500 down all strikes. 
# (and this value 500 must be a variable so i can change it later)
# 2-) similarly in bank nifty 1000 up and 1000 down all strikes. ( and this value 1000 must be a variable so i can change it 
#                                                                 later)
# 3-) stocks it should count 10% up and 10% down. ( and this value 10% must be a variable so i can change it later)

# CSV File with number of sheet for each strikes with below columns
# Date, Stock/Index, Expiry, RIGHT, Open Price, close Price, Current OI, Previous Minute OI, OI change, Percent Up/Down From 
# Last Minute OI, RIGHT, Open Price, close Price, Current OI, Previous Minute OI, OI change, Percent Up/Down From Last Minute OI
# 9:45:01, Nifty50, 20210527, CALL, 102.1, 90 (if not close yesterday keep0), 45555, 45000, +555, +7%, PUT, 52.8, 67 (if not 
#                                             close yesterday keep0), 35555, 35000, +555, +6%

In [130]:
from selenium import webdriver
import pandas as pd
import numpy as np

In [131]:
import requests
headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.1.1 Safari/605.1.15","Accept-Language": "en-gb","Accept-Encoding":"br, gzip, deflate","Accept":"test/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8","Referer":"http://www.google.com/"}
url = "https://www.nseindia.com/api/option-chain-indices?symbol=NIFTY"
page = requests.get(url,headers=headers)
page

<Response [200]>

In [132]:
nifty_margin = 500
bank_nifty_margin = 1000
stock_margin = 10

In [133]:
col_list = ['strikePrice','expiryDate','openInterest']
# data_format = ['Date','Stock/Index','Expiry','Right','Open Price','Close Price','Current OI','Previous Minute OI',
#                'OI Change','% Up/Down from Last Minute OI']

In [134]:
selected_margin = nifty_margin

In [135]:
import json
res = json.loads(page.text)['records']['data']

In [136]:
underlying_value = int(res[0]['PE']['underlyingValue'])
type(underlying_value)

int

In [170]:
data = [d.get('CE') for d in res]
data_cleaned = list(filter(None, data))
df_call = pd.DataFrame(data_cleaned)

data = [d.get('PE') for d in res]
data_cleaned = list(filter(None, data))
df_put = pd.DataFrame(data_cleaned)

df_put['expiryDate'] = pd.to_datetime(df_put['expiryDate'])
df_call['expiryDate'] = pd.to_datetime(df_call['expiryDate'])


df_put.sort_values(by=['expiryDate','strikePrice'],inplace=True)
df_call.sort_values(by=['expiryDate','strikePrice'],inplace=True)

df_put = df_put[col_list]
df_call = df_call[col_list]

In [171]:
df_put['strikePrice'] = df_put['strikePrice'].map(int)
df_call['strikePrice'] = df_call['strikePrice'].map(int)

In [172]:
# min, max = ((100-percentage)*0.01)*(df['underlyingValue'][0]), ((100+percentage)*0.01)*(df['underlyingValue'][0])
# df_percent = df.loc[(df['strikePrice']>=min) & (df['strikePrice']<=max)]
# df_percent

In [173]:
df_put = df_put[(df_put['strikePrice'] >= underlying_value-selected_margin) & (df_put['strikePrice']<=underlying_value+selected_margin)]
df_call = df_call[(df_call['strikePrice'] >= underlying_value-selected_margin) & (df_call['strikePrice']<=underlying_value+selected_margin)]
df_put['RIGHT'] = 'PUT'
df_call['RIGHT'] = 'CALL'

In [174]:
max_sp = df_call.strikePrice.max
min_sp = df_call.strikePrice.min

In [175]:
sp_range = df_put.strikePrice.unique().tolist()

In [178]:
for i in sp_range:
    temp_df_put = df_put[df_put['strikePrice']==i]
    temp_df_call = df_call[df_call['strikePrice']==i]

     strikePrice expiryDate  openInterest RIGHT
426        14700 2021-05-27         31764   PUT
427        14700 2021-06-03          2212   PUT
419        14700 2021-06-10           258   PUT
420        14700 2021-06-17             1   PUT
428        14700 2021-06-24         10369   PUT
421        14700 2021-07-01             0   PUT
422        14700 2021-07-08             0   PUT
423        14700 2021-07-15             0   PUT
424        14700 2021-07-22             0   PUT
425        14700 2021-07-29           202   PUT
     strikePrice expiryDate  openInterest RIGHT
405        14700 2021-05-27         10115  CALL
406        14700 2021-06-03           564  CALL
398        14700 2021-06-10             0  CALL
399        14700 2021-06-17             0  CALL
407        14700 2021-06-24           895  CALL
400        14700 2021-07-01             0  CALL
401        14700 2021-07-08             0  CALL
402        14700 2021-07-15             0  CALL
403        14700 2021-07-22             

     strikePrice expiryDate  openInterest RIGHT
577        15550 2021-05-27         12998  CALL
573        15550 2021-06-03           212  CALL
581        15550 2021-06-10             0  CALL
575        15550 2021-06-17             0  CALL
576        15550 2021-06-24           316  CALL
574        15550 2021-07-01             0  CALL
578        15550 2021-07-15             0  CALL
579        15550 2021-07-22             0  CALL
580        15550 2021-07-29            10  CALL
     strikePrice expiryDate  openInterest RIGHT
605        15600 2021-05-27           277   PUT
614        15600 2021-06-03            15   PUT
606        15600 2021-06-10             6   PUT
607        15600 2021-06-17             0   PUT
608        15600 2021-06-24           185   PUT
609        15600 2021-07-01             0   PUT
610        15600 2021-07-08             0   PUT
611        15600 2021-07-15             0   PUT
612        15600 2021-07-22             0   PUT
613        15600 2021-07-29             

In [184]:
temp1 = df_call
temp2 = df_put
# Ispe bharosa mat karna
temp1.merge(temp2,left_index=True,right_index=True,on='strikePrice')

,strikePrice,expiryDate_x,openInterest_x,RIGHT_x,expiryDate_y,openInterest_y,RIGHT_y
426,14800,2021-05-27,22670,CALL,2021-05-27,31764,PUT
436,14850,2021-05-27,1022,CALL,2021-05-27,7128,PUT
446,14900,2021-05-27,13897,CALL,2021-06-03,2771,PUT
457,14950,2021-05-27,2153,CALL,2021-05-27,7821,PUT
465,15000,2021-05-27,35065,CALL,2021-07-22,0,PUT
...,...,...,...,...,...,...,...
570,15500,2021-09-30,3,CALL,2021-07-22,0,PUT
467,15000,2021-12-30,5037,CALL,2021-05-27,39429,PUT
562,15500,2021-12-30,329,CALL,2021-07-08,0,PUT
468,15000,2022-03-31,0,CALL,2021-06-17,0,PUT


In [145]:
import pandas as pd
import xlsxwriter
df1 = pd.DataFrame({'Data': ['a', 'b', 'c', 'd']})

df2 = pd.DataFrame({'Data': [1, 2, 3, 4]})

df3 = pd.DataFrame({'Data': [1.1, 1.2, 1.3, 1.4]})

writer = pd.ExcelWriter('multiple.xlsx', engine='xlsxwriter')

df1.to_excel(writer, sheet_name='Sheet_1')

df2.to_excel(writer, sheet_name='Sheet_2')

df3.to_excel(writer, sheet_name='Sheet_3')

writer.save()